In [1]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import math
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [3]:
link='https://www.skokka.com/'


In [4]:
driver.get(link)

In [16]:
urlstoscrape = []

In [17]:
desnum=0
while desnum==0:
    html=BeautifulSoup(driver.page_source, 'html.parser')
    targets= html.find_all('p', class_='listing-title item-title')   
    for target in targets:
        try:
            link = target.find('a')['href']
            if link:
                urlstoscrape.append(link)
                
        except Exception as e:
            print('Nohay')


    if len(driver.find_elements(By.XPATH, '//a[@aria-label="Next"]'))>0:
        try:
            driver.find_element(By.XPATH, '//a[@aria-label="Next"]').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            time.sleep(2)
    else:
        desnum=1

In [19]:
titulo = []
numero = []
descripcion = [] 

In [20]:
urlstoscrape=pd.Series(urlstoscrape).drop_duplicates().tolist()

In [69]:
# Espera hasta que el elemento principal de la página esté presente antes de continuar
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
)

<selenium.webdriver.remote.webelement.WebElement (session="f081539e720ba5e2dac07276233cc367", element="f.39E2408F7495A1C73A190D407D83AEDC.d.F36829DE96855D82E119A70DB13FBE34.e.1212")>

In [21]:
for url in urlstoscrape:

    driver.get(url)
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
)
    page= BeautifulSoup(driver.page_source, 'html.parser')
    try:
        pretitle= page.find('h1', class_='main-title').text.strip()
        if pretitle:
            titulo.append(pretitle)
        else:
            titulo.append(None)
        
    except AttributeError:
        titulo.append(None)
    try:
        prenumero= page.find('button', id='phone-button').get('href')
        prenumero=prenumero.replace('tel:', '').strip()
        if prenumero:
            numero.append(prenumero)
        else:
            numero.append(None)
        
    except AttributeError:
        numero.append(None)
    try:
        predescripcion= page.find('div', class_='col service-detail').text.strip().replace('\n', ' ').replace('\r', '').replace('  ', ' ').strip()
        if predescripcion:
            predescripcion.append(predescripcion)
        else:
            descripcion.append(None)
        
    except AttributeError:
        descripcion.append(None)


In [27]:
sample=({
    'titulo': titulo,
    'numero': numero,
    'descripcion': descripcion
})

In [28]:
sample

{'titulo': ['NATALIA  EXOTICA',
  'ALISON COLOMBIANA JUGUETONA',
  'VALENTINA ATREVIDA MUY elegante e implicada',
  'SOFIA RUBIA ALTA DELGADA  COMPAÑIA DE LUJO',
  'ISABELLA EXPERIMENTADA FOGOSA',
  'Elissa la más complaciente.',
  'GALA SEXY Y ARDIENTE , NUEVA EN LA ZONA',
  'AMIGUITAS LATINAS EN LA ZONA DE VIA JULIA',
  'Nov. Servicios sin prisas 50 media hora correte en mi boca',
  'MASAJES RELAJATES ORIENTALES',
  'Jovencita de 20 años sensual y juguetona',
  'Masajistas eroticas, siempre disponibles',
  'ATENEA VENEZOLANA MUY COMPLACIENTE',
  'SALIDAS,FIESTA Y SEXO A TOPE EN TORRASA 24/7',
  'SEXY AMIGUITAS, DIVERTIDAS Y FIESTERAS DISPONIBLES 24/7 EN COLLBLANC',
  'GRÀCIA SEXO',
  'Soy  colombiana divertida  atrevida cariñosa puro fuego  todita disponible  para complacer tus caprichos',
  'HERMOSA CHICA JOVEN 😈🔥',
  'Beatriz CON GANAS DE TODO',
  'DANIELA latina exuberante',
  'RENATA BRASILEÑA MUY CANDENTE',
  'MELANIE COLOMBIANA DULCE Y SEDUCTORA',
  'LIZZY CALIENTE',
  'Pamela 

In [29]:
sample = pd.DataFrame(sample).drop_duplicates()

In [30]:
sample

,titulo,numero,descripcion
0,NATALIA EXOTICA,640942699,None
1,ALISON COLOMBIANA JUGUETONA,640262571,None
2,VALENTINA ATREVIDA MUY elegante e implicada,640262571,None
3,SOFIA RUBIA ALTA DELGADA COMPAÑIA DE LUJO,640262571,None
4,ISABELLA EXPERIMENTADA FOGOSA,640262571,None
...,...,...,...
341,💛LATINA COMPLACIENTE 💛,666778920,None
342,"Catalana, tus deseos viven en mi piel",622926380,None
343,STEFANIA COMPLACIENTE🌻NUEVA,666778920,None
344,🌟LLEGANDO A TU CUIDAD 💘,666778920,None


In [31]:
sample.to_excel('skokkaBarcelona_Mujeres.xlsx', index=False)
sample.to_csv('skokkaBarcelona_Mujeres.csv', index=False)